## Publisher Spreadsheets

In [ ]:
import importlib
import os

import parse_publisher_sheets as Publishers
importlib.reload(Publishers)

In [ ]:
all_publ_dicts = Publishers.main()

In [ ]:
publ_isbns_set = Publishers.make_set_all_isbns(all_publ_dicts)

## Bookstore csv

In [ ]:
import re
import parse_bookstore_csv as Bookstore

ISBNregex = re.compile(r'(\b\d{13}\b)|(\b\d{9}[\d|X]\b)')

bookstore_csv_as_list = Bookstore.cleanup_original_text('output/Summerbookstore2017.csv')

In [ ]:
bookstore_isbns = {row[7].replace('-', ''): row for row in bookstore_csv_as_list if ISBNregex.match(row[7].replace('-', ''))}

## Symphony isbns


In [ ]:
ISBNregex = re.compile(r'(\b\d{13}\b)|(\b\d{9}[\d|X]\b)')

def findISBNs(filepath, filename):
    isbns = []
    full_filepath = os.path.join(filepath, filename)
    with open(full_filepath, "r", encoding="utf-8", errors="surrogateescape") as isbn_lines:
        read_data = isbn_lines.readlines()
    for line in read_data:
        isbns.extend(ISBNregex.findall(line))
    stripped = set()
    stripped = {isbn.replace('-', '') for tuple_group in isbns for isbn in tuple_group if isbn}
    return stripped

Symphony_isbns = findISBNs('output', 'symphony.txt')

## Worldcat expanding bookstore isbns

In [ ]:
import findEtextbooks as Combine

importlib.reload(Combine)

In [ ]:
sets_of_similar_isbns = list()
for isbn in set(bookstore_isbns.keys()):
    set_of_similar_isbns = Combine.find_similar_isbns(isbn)
    sets_of_similar_isbns.append(set_of_similar_isbns)
xCourseISBNs = Combine.flatten_set_of_sets(sets_of_similar_isbns)

In [ ]:
def show_similar_isbns(isbn):
    for similar_set in sets_of_similar_isbns:
        if isbn in similar_set:
            return similar_set
    

## Course Catalog

In [ ]:
import parse_course_catalog as CourseCatalog

course_file = 'course_listings/2017/Summer_2017/MUSIC3S17A.txt'

course_nts, season_dept = CourseCatalog.parse_course_listing_texts(course_file)

In [ ]:
def lookup_course_catalog_item(bookstore_item_dict, year, season):
    filepath = os.path.join('course_listings', year, season)
    if not os.path.isdir(filepath):
        print('year must be among: {}'.format(', '.join(i for i in os.listdir(course_listings))))
    for root, dirs, files in os.walk(filepath):
        for file in files:
            sought_dept = bookstore_item_dict.get('Dept/Course').split(' ')[0]
            if sought_dept in file:
                filepath = os.path.join(root, file)
                course_nts, season_dept = CourseCatalog.parse_course_listing_texts(filepath)
                return [course_nt for course_nt in course_nts
                        if course_nt.abbr_num.replace(' ', '') == bookstore_item_dict['Dept/Course'].replace(' ', '')]

In [ ]:
bookstore_item = {'Publisher': 'TAYLOR', 
                  'RcCd': 'RQ',
                  'STS': '',
                  'Author': 'CORNELIUS',
                  'empty1': '',
                  'Title': 'MUSIC   -TEXT',
                  'ISBN': '978-0-13-601750-9',
                  'Dept/Course': 'MUS 1751',
                  'empty2': '',
                  'Professor': 'Perry',
                  'Section': '3'}

year = '2017'
season = 'Summer_2017'

lookup_course_catalog_item(bookstore_item, year, season)

## Ebsco Discovery

In [ ]:
import ebsco_discovery_functions as Discovery

In [ ]:
import json

response = Discovery.main('9780136017509')
response_json = json.dumps(json.loads(response), sort_keys=True, indent=2)
print(response_json)

## Show all we know about an isbn

In [ ]:
isbn = '9780136017509'


print('bookstore isbn:', isbn)
print('similar isbns:', show_similar_isbns(isbn), '\n')
if show_similar_isbns(isbn):
    for isbn in show_similar_isbns(isbn):
        is_snippet_in_publisher_files_field(isbn, 'isbn')
        print('\n')
        pub_item = publ_isbns_set.get(isbn)
        if pub_item:
            print("From Publisher websites:", pub_item, '\n')
bookstore_item = bookstore_isbns.get(isbn)
if bookstore_item:
    bookstore_headers = ('Dept/Course', 'Section', 'empty1', 'Professor',
   'Author', 'Title', 'empty2', 'ISBN', 'Publisher', 'RcCd', 'STS')
    bookstore_item_dict = {k: v for k, v in zip(bookstore_headers, bookstore_item)}
    print("From the B&N Bookstore csv:", bookstore_item_dict, "\n")
    print('\n')
    print("From Course Catalog:", lookup_course_catalog_item(bookstore_item_dict), '\n')
print('\n')

In [ ]:
def is_snippet_in_publisher_files_field(snippet, field):
    for filename, sheets in all_publ_dicts.items():
        for sheetname, textbooks in sheets.items():
            for textbook, descriptor in textbooks.items():
                for attribute, value in descriptor.items():
                    if field.lower() in attribute.lower():
                        if isinstance(value, str):
                            if snippet.lower() in value.lower():
                                print(descriptor)
    print('no such "{}" in an Publishers spreadsheetfield with "{}" in the name'.format(snippet, field))

is_snippet_in_publisher_files_field('9780136017509', 'isbn')

In [ ]:
def multiple_isbns():
    pub_expanded_isbns = []
    for filename, sheets in all_publ_dicts.items():
        for sheetname, textbooks in sheets.items():
            for textbook, descriptor in textbooks.items():
                alternate_isbns = [i for i in descriptor if 'isbn' in i.lower()]
                if len(alternate_isbns) > 1:
                    pub_expanded_isbns.append(alternate_isbns)
    return pub_expanded_isbns

pub_expanded_isbns = multiple_isbns()
                        